# PDSH
## Chapter 3
...

### example: recipe database

In [5]:
import numpy as np
import pandas as pd
from io import StringIO

try:
    recipes = pd.read_json(r"C:\Users\KestriC\Desktop\PDSH\recipeitems-latest.json")
except ValueError as e:
    print("ValueError:", e) # each line is json

ValueError: Trailing data


In [2]:
with open(r"C:\Users\KestriC\Desktop\PDSH\recipeitems-latest.json") as f:
    line = f.readline()
pd.read_json(StringIO(line)).shape # each line is a json

(2, 12)

In [7]:
# construct string representation of all json entries

# read entire file into py array
with open(r"C:\Users\KestriC\Desktop\PDSH\recipeitems-latest.json", "r", errors="ignore") as f:
    # extract each line
    data=(line.strip() for line in f)
    # make each line element of a list
    data_json = "[{0}]".format(",".join(data))
# read results as json
recipes = pd.read_json(StringIO(data_json))
recipes.shape

(173278, 17)

In [8]:
# check one row
recipes.iloc[0]

_id                                {'$oid': '5160756b96cc62079cc2db15'}
name                                    Drop Biscuits and Sausage Gravy
ingredients           Biscuits\n3 cups All-purpose Flour\n2 Tablespo...
url                   http://thepioneerwoman.com/cooking/2013/03/dro...
image                 http://static.thepioneerwoman.com/cooking/file...
ts                                             {'$date': 1365276011104}
cookTime                                                          PT30M
source                                                  thepioneerwoman
recipeYield                                                          12
datePublished                                                2013-03-11
prepTime                                                          PT10M
description           Late Saturday afternoon, after Marlboro Man ha...
totalTime                                                           NaN
creator                                                         

In [10]:
# closer look at the ingredients
recipes.ingredients.str.len().describe()

count    173278.000000
mean        247.406866
std         147.065429
min           0.000000
25%         151.000000
50%         224.000000
75%         317.000000
max        9083.000000
Name: ingredients, dtype: float64

In [15]:
# how many breakfast recipes
recipes.ingredients.str.contains("[Bb]reakfast").sum()

233

In [18]:
import re
# recipe recommendation system

# list of common ingredients, search whether they are in ingr. list
spice_list = ["salt", "pepper", "oregano", "sage", "parsley", "rosemary", "tarragon", "thyme", "paprika", "cumin"]

# build bool df
spice_df = pd.DataFrame(dict((spice, recipes.ingredients.str.contains(spice, re.IGNORECASE)) for spice in spice_list))
spice_df.head()

,salt,pepper,oregano,sage,parsley,rosemary,tarragon,thyme,paprika,cumin
0,False,False,False,True,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,True,True,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False


In [20]:
# find recipe using parsley, paprika and tarragon (query() method)
selection = spice_df.query("parsley & paprika & tarragon")
len(selection) # how many recipes were found with those ingredients

10

In [22]:
# names of the recipes
recipes.name[selection.index]

2069      All cremat with a Little Gem, dandelion and wa...
74964                         Lobster with Thermidor butter
93768      Burton's Southern Fried Chicken with White Gravy
113926                     Mijo's Slow Cooker Shredded Beef
137686                     Asparagus Soup with Poached Eggs
140530                               Fried Oyster Poâ€™boys
158475                Lamb shank tagine with herb tabbouleh
158486                 Southern fried chicken in buttermilk
163175            Fried Chicken Sliders with Pickles + Slaw
165243                        Bar Tartine Cauliflower Salad
Name: name, dtype: object

## Working with Time Series
### Dates and Times in Python
#### native python: datetime & dateutil

In [23]:
# built-in datetime module & dateutil module
from datetime import datetime

# manually build a date
datetime(year=2015, month=7, day=4)

datetime.datetime(2015, 7, 4, 0, 0)

In [25]:
from dateutil import parser

# parse date from string
date = parser.parse("4th of July, 2015")
date

datetime.datetime(2015, 4, 7, 0, 0)

In [29]:
date = parser.parse("4/7/2015")
date

datetime.datetime(2015, 4, 7, 0, 0)

In [30]:
# from datetime obj it is possible to print eg day of week
date.strftime("%A")

'Tuesday'

#### numpy datetime64

In [31]:
# encodes dates as 64-bit ints
import numpy as np
date = np.array("2015-07-04", dtype=np.datetime64)
date

array('2015-07-04', dtype='datetime64[D]')

In [32]:
# apply vectorized operations
date + np.arange(12)

array(['2015-07-04', '2015-07-05', '2015-07-06', '2015-07-07',
       '2015-07-08', '2015-07-09', '2015-07-10', '2015-07-11',
       '2015-07-12', '2015-07-13', '2015-07-14', '2015-07-15'],
      dtype='datetime64[D]')

In [36]:
# nanosecond based time (frequently used)
np.datetime64("2015-07-04 12:59:59.50", "ns")

numpy.datetime64('2015-07-04T12:59:59.500000000')

#### dates and times in pandas: best of both worlds

In [37]:
# pd creates timestamp obj
import pandas as pd
date = pd.to_datetime("4th of July, 2015")
date

Timestamp('2015-07-04 00:00:00')

In [38]:
# extract day of week
date.strftime("%A")

'Saturday'

In [39]:
# np style vectorized operations
date + pd.to_timedelta(np.arange(12), "D")

DatetimeIndex(['2015-07-04', '2015-07-05', '2015-07-06', '2015-07-07',
               '2015-07-08', '2015-07-09', '2015-07-10', '2015-07-11',
               '2015-07-12', '2015-07-13', '2015-07-14', '2015-07-15'],
              dtype='datetime64[ns]', freq=None)

## Pandas Time Series: indexing by time

In [42]:
# ser obj indexed by timestamp
index = pd.DatetimeIndex(["2014-07-04", "2014-08-04", "2015-07-04", "2015-08-04"])
data = pd.Series([0, 1, 2, 3], index=index)
data

2014-07-04    0
2014-08-04    1
2015-07-04    2
2015-08-04    3
dtype: int64

In [44]:
data["2014-07-04": "2015-07-04"] # slicing

2014-07-04    0
2014-08-04    1
2015-07-04    2
dtype: int64

In [47]:
# special timestamp indexing operations
data["2015"]

2015-07-04    2
2015-08-04    3
dtype: int64

In [48]:
dates = pd.to_datetime([datetime(2015, 7, 3), "4th of July, 2015", "2015-Jul-6", "07-07-2015", "20150708"]) # parses to DatetimeIndex
dates

DatetimeIndex(['2015-07-03', '2015-07-04', '2015-07-06', '2015-07-07',
               '2015-07-08'],
              dtype='datetime64[ns]', freq=None)

In [49]:
dates - dates[0] # this creates a TimedeltaIndex

TimedeltaIndex(['0 days', '1 days', '3 days', '4 days', '5 days'], dtype='timedelta64[ns]', freq=None)

In [50]:
pd.date_range("2015-07-03", "2015-07-10") # creates datetime seq

DatetimeIndex(['2015-07-03', '2015-07-04', '2015-07-05', '2015-07-06',
               '2015-07-07', '2015-07-08', '2015-07-09', '2015-07-10'],
              dtype='datetime64[ns]', freq='D')

In [51]:
pd.date_range("2015-07-03", periods=8, freq="H") # range of hourly timestamps

DatetimeIndex(['2015-07-03 00:00:00', '2015-07-03 01:00:00',
               '2015-07-03 02:00:00', '2015-07-03 03:00:00',
               '2015-07-03 04:00:00', '2015-07-03 05:00:00',
               '2015-07-03 06:00:00', '2015-07-03 07:00:00'],
              dtype='datetime64[ns]', freq='H')

In [56]:
pd.period_range("2015-07", periods=8, freq="M")

PeriodIndex(['2015-07', '2015-08', '2015-09', '2015-10', '2015-11', '2015-12',
             '2016-01', '2016-02'],
            dtype='period[M]')

In [58]:
pd.timedelta_range(0, periods=10, freq="H") # durations increasing by an hour

TimedeltaIndex(['0 days 00:00:00', '0 days 01:00:00', '0 days 02:00:00',
                '0 days 03:00:00', '0 days 04:00:00', '0 days 05:00:00',
                '0 days 06:00:00', '0 days 07:00:00', '0 days 08:00:00',
                '0 days 09:00:00'],
               dtype='timedelta64[ns]', freq='H')

### frequencies and offsets

In [59]:
pd.timedelta_range(0, periods=9, freq="2H30T") # frequency of 2 hours 30 mins

TimedeltaIndex(['0 days 00:00:00', '0 days 02:30:00', '0 days 05:00:00',
                '0 days 07:30:00', '0 days 10:00:00', '0 days 12:30:00',
                '0 days 15:00:00', '0 days 17:30:00', '0 days 20:00:00'],
               dtype='timedelta64[ns]', freq='150T')